Крч тут такая тема, бот чисто прототипный(лень было делать все красиво, потом допилю красоту, т.к все же будет переделано)

In [1]:
# Импортирование всех необходимых библиотек
import pandas as pd
import numpy as np
import pickle
from sklearn.pipeline import FeatureUnion
import re
import json
import joblib
from geopy.geocoders import Nominatim
from datetime import datetime
import geocoder
import warnings
pd.options.mode.chained_assignment = None 
warnings.filterwarnings("ignore")

In [2]:
path = 'Data/Data.csv'

from nltk.corpus import stopwords
stop_words = stopwords.words("russian")

In [3]:
# Трансформаторы для получения данных
from sklearn.base import BaseEstimator, TransformerMixin
# Конвеер для объединения шагов выполнения классификации
from sklearn.pipeline import Pipeline
# Функция стандартизации числовых значений
from sklearn.preprocessing import StandardScaler
# Функция преобразования текстовых данных под модель
from sklearn.feature_extraction.text import TfidfVectorizer


# Класс необходимый для получения и работы с текстовыми признаками
class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.key]
    
# Класс необходимый для получения работы с числовыми признаками
class NumericalSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.key]]
    
# Стеммер, работающий с русским языком
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian") 

# Функция стеммизации данных для образования однокоренных слов
def stemming_tokenizer(text):
    text = re.split('\W+', text)
    text = [stemmer.stem(word) for word in text]
    return text

import nltk
from nltk.corpus import stopwords


# Функция для очистки столбца 
def clean_data(df):
    df = df.str.lower()
    df.replace(r'[^а-яА-Яa-zA-Z0-9]', ' ', regex=True, inplace=True)
    df = df.str.strip()
    stop_words = stopwords.words("russian")
    df = df.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    return df

In [4]:
# Векторайзер
feat = joblib.load('feat.z')
#Текстовый трансформатор
num = joblib.load('num.z')

feat = FeatureUnion([('feat', feat),
                    ("num", num)])

In [5]:
def read(pred=False):
    global df
    g = geocoder.location(input('Где это произошло?'))
    print("Вы знаете точные координаты")
    answer = input()
    if (str(answer) == "Да" )|( str(answer) == 'да'):
        lat = float(input("Введите latitude"))
        lon = float(input("Введите lonitude"))
    elif g.latitude == None:
        print(
            "Ошибка в геокаддировании адреса, введите пожалуйста ширину и долготу"
        )
        lat = float(input("Введите latitude"))
        lon = float(input("Введите lonitude"))
    else:
        lat = g.latitude
        lon = g.longitude
    print("Это сегодня произошло? ")
    answer = input()
    if answer == "Да":
        date = datetime.today()
    else:
        date = datetime.strptime(input("Введи дату в формате 18.08.21 18:00"),
                                 '%d.%m.%y %H:%M')
    gibddid = input('Введите id происшествия, если есть ')
    region = input('Введите region ')
    parent_region = input('Введите parent region ')

    feat = input(
        "Опишите ДТП (Что находилось рядом, освещение, состояние дороги)")
    inj = int(input('Сколько пострадавших? Ответ в числовом формате'))
    dead = int(input('Сколько человек погибло? Ответ в числовом формате'))
    intraction = int(4 * int(dead) + int(inj))
    df_concat = pd.DataFrame([[
        gibddid, region, parent_region, date, g.location, lat, lon, feat, inj,
        dead, intraction
    ]],
                             columns=[
                                 'gibddid', 'region', 'parent_region', 'date',
                                 'address', 'lat', 'lon', 'feat', 'inj_count',
                                 'dead_count', 'interaction'
                             ])

    if pred:
        return df_concat
    df = pd.concat([df, df_concat], axis=0
                  )
    print('Спасибо за ваше участие в опросе!')
    df.reset_index()
    

In [6]:
# Импорт модели классификации
model = joblib.load('my_model.z')
df=pd.read_csv('Data/test.csv', sep='\t', encoding='UTF-16')

u="\nПользователь:"
b="\nБот:"

# Приветствие
welcome=("Вас приветствует Бот. Существует несколько команд:\n"
"!exit - Выход из приложения\n"
"!predict - Вывод блока компетенции по входным параметрам\n"
"!datacollecting - Вывод компетенций по входным параметрам\n"
"!help - Вывод существующих команд\n"
"!skolko - узнать сколько участков\n")

print(b, welcome)



Бот: Вас приветствует Бот. Существует несколько команд:
!exit - Выход из приложения
!predict - Вывод блока компетенции по входным параметрам
!datacollecting - Вывод компетенций по входным параметрам
!help - Вывод существующих команд
!skolko - узнать сколько участков



In [ ]:
# Непосредственно сам бот
while True:
    try:
        input_text=input(u).lower()
        # Команды бота
        if any(word in input_text for word in ("завершить", "выход", "конец")):
            print(b, "Завершение работы ")
        if any(word in input_text for word in ("help", "умеешь", "команды")):
            print(b,welcome )
        if any(word in input_text for word in ("сколько", "городе", "проеду")):
            gor = input("Введите город, который вас интересует ")
            level =input("введите уровень опасности ")
            df = df[df['y'] == level]
            print((df['features.properties.region']==gor).sum())
            
        if any(word in input_text for word in ("какой", "предскажи", "узнать")):
         # Ввод данных
            df = read(True)
            X = feat.transform(df[['feat','interaction']])
            if model.predict(X) ==[1]:
                print('Участок средней опасности')
            if model.predict(X) ==[0]:
                print('Очень опасный участок')
            if model.predict(X) ==[2]:
                print('Добрый участок')
            print(model.predict(X))
        if input_text == "!datacollecting":
            read()
        if input_text == "!exit":
            break
    except:
        print("Что-то погло не так")


In [ ]:
import pandas as pd

import pandas as pd
import json
from geopy.geocoders import Nominatim
from datetime import datetime
import geocoder
path = 'Data/Data.csv'
df = pd.DataFrame(columns=[
    'gibddid', 'region', 'parent_region', 'date', 'address', 'lat', 'lon',
    'feat', 'inj_count', 'dead_count', 'interaction'
])


def read(pred=False):
    global df
    g = geocoder.location(input('Где это произошло?'))
    print("Вы знаете точные координаты")
    answer = input()
    if (str(answer) == "Да" )|( str(answer) == 'да'):
        lat = float(input("Введите latitude"))
        lon = float(input("Введите lonitude"))
    elif g.latitude == None:
        print(
            "Ошибка в геокаддировании адреса, введите пожалуйста ширину и долготу"
        )
        lat = float(input("Введите latitude"))
        lon = float(input("Введите lonitude"))
    else:
        lat = g.latitude
        lon = g.longitude
    print("Это сегодня произошло? ")
    answer = input()
    if answer == "Да":
        date = datetime.today()
    else:
        date = datetime.strptime(input("Введи дату в формате 18.08.21 18:00"),
                                 '%d.%m.%y %H:%M')
    gibddid = input('Введите id происшествия, если есть ')
    region = input('Введите region ')
    parent_region = input('Введите parent region ')

    feat = input(
        "Опишите ДТП (Что находилось рядом, освещение, состояние дороги)")
    inj = int(input('Сколько пострадавших? Ответ в числовом формате'))
    dead = int(input('Сколько человек погибло? Ответ в числовом формате'))
    intraction = int(4 * int(dead) + int(inj))
    df_concat = pd.DataFrame([[
        gibddid, region, parent_region, date, g.location, lat, lon, feat, inj,
        dead, intraction
    ]],
                             columns=[
                                 'gibddid', 'region', 'parent_region', 'date',
                                 'address', 'lat', 'lon', 'feat', 'inj_count',
                                 'dead_count', 'interaction'
                             ])

    if pred:
        return df_concat
    df = pd.concat([df, df_concat], axis=0
                  )
    print('Спасибо за ваше участие в опросе!')
    df.reset_index()
    